In [13]:
import pandas as pd
import json
import numpy as np

def parse_json_column(column):
    """
    Safely parse a column that contains JSON-like strings or dictionaries.
    If the column is empty, null, or invalid, return an empty dictionary.
    This function handles cases where the column might already be a dictionary or a JSON string.
    """
    # Check if the column is empty, null, or already an empty dictionary/list
    if pd.isna(column) or column == {} or column == []:
        return {}
    # If the column is already a dictionary, return it as is
    if isinstance(column, dict):
        return column
    # If the column is a string, try to parse it as JSON
    if isinstance(column, str):
        try:
            # Replace single quotes with double quotes to ensure valid JSON format
            column = column.replace("'", '"')
            # Unescape any escaped characters in the string
            column = bytes(column, "utf-8").decode("unicode_escape")
            # Parse the JSON string into a Python dictionary
            return json.loads(column)
        except (ValueError, SyntaxError, json.JSONDecodeError):
            # If parsing fails, return an empty dictionary
            return {}
def flatten_and_merge(df, column, prefix):
    """
    This function takes a DataFrame, parses a specified JSON column, flattens it into multiple columns,
    and merges the flattened data back into the original DataFrame.
    The flattened columns are prefixed with the given prefix for clarity.
    """
    # Create a new column name for the parsed JSON data
    parsed_column = f"{column}_parsed"
    # Parse the JSON column using the parse_json_column function
    df[parsed_column] = df[column].apply(parse_json_column)
    # Flatten the parsed JSON column into separate columns
    flattened_df = pd.json_normalize(df[parsed_column])
    # Add a prefix to the flattened columns to avoid naming conflicts
    flattened_df = flattened_df.add_prefix(f"{prefix}.")
    # Reset the indices of both DataFrames to ensure proper alignment during concatenation
    flattened_df = flattened_df.reset_index(drop=True)
    df = df.reset_index(drop=True)
    # Merge the flattened DataFrame back into the original DataFrame
    df = pd.concat([df, flattened_df], axis=1)
    # Drop the intermediate parsed column as it's no longer needed
    df = df.drop(columns=[parsed_column])
    return df
def explode_and_flatten(df, column, prefix):
    """
    This function takes a DataFrame, explodes a specified column (which contains lists or JSON-like structures),
    flattens the exploded values into separate columns, and merges the result back into the original DataFrame.
    The flattened columns are prefixed with the given prefix for clarity.
    """
    # Explode the column to create one row for each element in the list/JSON structure
    df = df.explode(column)
    # Reset the index after exploding to ensure all indices are unique and aligned
    df = df.reset_index(drop=True)
    # Flatten the exploded column into separate columns
    flattened_df = pd.json_normalize(df[column])
    # Add a prefix to the flattened columns to avoid naming conflicts
    flattened_df = flattened_df.add_prefix(f"{prefix}.")
    # Reset the index of the flattened DataFrame to ensure proper alignment
    flattened_df = flattened_df.reset_index(drop=True)
    # Merge the flattened DataFrame back into the original DataFrame
    df = pd.concat([df, flattened_df], axis=1)
    return df



#Execution part

# Load the CSV file into a DataFrame
df = pd.read_csv('C:/Users/lesze/classifications.csv', sep=',') #file with Zooniverse classifications
df = df.drop(columns=['gold_standard','expert'])
print('data loaded')

df = df[(df.workflow_name == 'Are Clouds Present?')]
df['created_at'] = pd.to_datetime(df['created_at']).dt.tz_localize(None)
df = df[(df.created_at > '2024-10-01')]

# Process the 'metadata' column: parse, flatten, and merge it back into the DataFrame
df = flatten_and_merge(df, 'metadata', 'metadata')

df['metadata.subject_dimensions']= df['metadata.subject_dimensions'].str.get(0)

df[['metadata.subject_dimensions.clientWidth',\
    'metadata.subject_dimensions.clientHeight',\
    'metadata.subject_dimensions.naturalWidth',\
    'metadata.subject_dimensions.naturalHeight']]\
       = df['metadata.subject_dimensions']\
       .astype(str)\
       .str.replace("{'clientWidth':","")\
       .str.replace(" 'clientHeight': ","")\
       .str.replace(" 'naturalWidth': ","")\
       .str.replace(" 'naturalHeight': ","")\
       .str.replace("}","")\
       .str.split(",", expand=True)

df['subject_data.orbit'] = df['subject_data'].astype(str)\
              .str.split('"Image_Orbit":"', expand=True)[1]\
              .str.split('"',expand=True)[0]
df['subject_data.Ls'] = df['subject_data'].astype(str)\
              .str.split('"Angle":"', expand=True)[1]\
              .str.split('"',expand=True)[0]              
df['subject_data.Angle'] = df['subject_data'].astype(str)\
              .str.split('"Angle":"', expand=True)[1]\
              .str.split('"',expand=True)[0]
df['subject_data.Binning'] = df['subject_data'].astype(str)\
              .str.split('"Binning":"', expand=True)[1]\
              .str.split('"',expand=True)[0]   


df['annotations_0'] = df['annotations'].apply(json.loads).apply(pd.Series)[0]
df = flatten_and_merge(df, 'annotations_0', 'annotations_0')
df['annotations_1'] = df['annotations'].apply(json.loads).apply(pd.Series)[1]
df = flatten_and_merge(df, 'annotations_1', 'annotations_1')

# Explode and flatten the 'annotations.1.value' column
df = explode_and_flatten(df, 'annotations_1.value', 'annotations_1.value')

df['orbit+username'] = df['subject_data.orbit'].astype(str) + '+' + df['user_name'].astype(str)
df['user_orbit_counts'] = df.groupby('orbit+username')['orbit+username'].transform('count')
df['rank'] = np.where(df['user_orbit_counts'] < 20, 'student',
             np.where(df['user_orbit_counts'] < 100, 'advanced', 'elite'))

df['weight'] = np.where(df['user_orbit_counts'] < 20, 1,
              np.where(df['user_orbit_counts'] < 100, 2, 3))

#unnecassary columns drop
df = df.drop(columns=['annotations_0','annotations_0.task','metadata','annotations',\
                      'subject_data','metadata.source',\
                      'metadata.live_project','metadata.subject_dimensions','annotations_1.task','annotations_1.value'])


# Save the processed DataFrame to different file formats for further use
# 1. Save to a CSV file with ";" as the delimiter
# This format is useful for sharing with tools that require semicolon-separated values
#df.to_csv('processed_data.csv', sep=';', index=False)
# 2. Save to a Parquet file
# Parquet is a columnar storage format optimized for performance and storage efficiency
#df.to_parquet('processed_data.parquet', index=False)
# 3. Save to an Excel file



print(df.shape)
print('writing to excel files')
df_no_clouds = df[(df['annotations_0.value'] == 'No')]
print(f'No clouds file shape: {df_no_clouds.shape}')
df_no_clouds.to_excel('C:/knime/no_clouds.xlsx', index=False)

df_broken_images = df[(df['annotations_0.value'] == 'The image is too broken up to see anything (bad data)')]
print(f'No clouds file shape: {df_broken_images.shape}')
df_broken_images.to_excel('C:/knime/broken_images.xlsx', index=False)
df_cloudy_images = df[(df['annotations_0.value'].str.contains('Yes'))]
print(f'No clouds file shape: {df_cloudy_images.shape}')
#df_cloudy_images.to_excel('C:/knime/cloudy_images.xlsx', index=False)

print('done')

data loaded
No clouds file shape: (113845, 50)
done


In [23]:
#some infotainment about the users

deduped_df = df_cloudy_images.drop_duplicates(subset=['user_name'], keep='first')

# 2. Group by rank and count users
rank_counts = (
    deduped_df
    .groupby('rank', observed=True)  # observed=True handles categorical data efficiently
    .size()
    .reset_index(name='user_count')
)

# 3. Sort by weight (optional)
rank_counts = rank_counts.sort_values('rank', key=lambda x: x.map({'student':1, 'advanced':2, 'elite':3}))

# Show results
print("\nUser Count by Rank:")
print(rank_counts.to_string(index=False))


User Count by Rank:
    rank  user_count
 student        3292
advanced          38
   elite           3
